<a href="https://colab.research.google.com/github/Annamjohn/Airline-Sentiment-Analysis/blob/main/Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the dependencies
import numpy as np
import pandas as pd
import re #pattern matching, search through data
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data Processing

In [ ]:
#loading data from csv file to pd DF

airline_data=pd.read_csv('/content/Airline-Sentiment.csv', encoding= 'ISO-8859-1')

In [ ]:
#no. of rows and columns
airline_data.shape

(14640, 20)

In [ ]:
airline_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2/24/15 11:14,5.703010e+17,NaN,Pacific Time (US & Canada)


In [ ]:
airline_data.isnull().sum() # sum of no. of null vals in each column



_unit_id                            0
_golden                             0
_unit_state                         0
_trusted_judgments                  0
_last_judgment_at                  56
airline_sentiment                   0
airline_sentiment:confidence        0
negativereason                   5462
negativereason:confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_id                            0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [ ]:
# twitter_data['content'] = twitter_data['content'].fillna('')
#twitter_data.isnull().sum()

In [ ]:
airline_data['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

**Stemming**

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):

    # Remove unwanted patterns
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # ^ =>not i.e. replace all non-alphabetic characters in content with a space
  stemmed_content= stemmed_content.lower()
  stemmed_content= stemmed_content.split() # Split the string into a list of words based on whitespace
  stemmed_content= [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content= ' '.join(stemmed_content) # Joins the list of stemmed words back into a single string for each tweet, with words separated by spaces.

  return stemmed_content

In [ ]:
airline_data['stemmed_content'] = airline_data['text'].apply(stemming)

In [ ]:
airline_data= airline_data[['stemmed_content','airline_sentiment']]
airline_data.head()

,stemmed_content,airline_sentiment
0,virginamerica dhepburn said,neutral
1,virginamerica plu ad commerci experi tacki,positive
2,virginamerica today must mean need take anoth ...,neutral
3,virginamerica realli aggress blast obnoxi ente...,negative
4,virginamerica realli big bad thing,negative


In [ ]:
print(airline_data['stemmed_content'])

0                              virginamerica dhepburn said
1               virginamerica plu ad commerci experi tacki
2        virginamerica today must mean need take anoth ...
3        virginamerica realli aggress blast obnoxi ente...
4                       virginamerica realli big bad thing
                               ...                        
14635          americanair thank got differ flight chicago
14636    americanair leav minut late flight warn commun...
14637    americanair pleas bring american airlin blackb...
14638    americanair money chang flight answer phone su...
14639    americanair ppl need know mani seat next fligh...
Name: stemmed_content, Length: 14640, dtype: object


In [ ]:
print(airline_data['airline_sentiment'])

0         neutral
1        positive
2         neutral
3        negative
4        negative
           ...   
14635    positive
14636    negative
14637     neutral
14638    negative
14639     neutral
Name: airline_sentiment, Length: 14640, dtype: object


In [ ]:
# seperate data and label

X= airline_data['stemmed_content'].values
Y=airline_data['airline_sentiment'].values

In [ ]:
print(X) # 1st 3 and last 3

['virginamerica dhepburn said'
 'virginamerica plu ad commerci experi tacki'
 'virginamerica today must mean need take anoth trip' ...
 'americanair pleas bring american airlin blackberri'
 'americanair money chang flight answer phone suggest make commit'
 'americanair ppl need know mani seat next flight plz put us standbi peopl next flight']


In [ ]:
print(Y)

['neutral' 'positive' 'neutral' ... 'neutral' 'negative' 'neutral']


In [ ]:
#Split dataset into train and test set
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.1, stratify= Y, random_state=42)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(14640,) (13176,) (1464,)


In [ ]:
print(X_train)

['americanair cancel flightl one'
 'jetblu flight boston delay due weather tampa'
 'unit thank appreci respons' ... 'jetblu enough stop fli jetblu'
 'southwestair thank make good poteettj cancel flightl flight'
 'usairway often airlin easi work need chang flight time attend mke made day']


In [ ]:
print(X_test)

['usairway bad weather mean bad servic'
 'southwestair let send dm follow lovedflyingwiththem notanymor wrong badpolici'
 'jetblu flight bo tampa takeoff min late flight' ...
 'southwestair kirkwoodtig hmmm caribbean sound http co aay avg b ld warm thank'
 'americanair hi flight dfw cancel flightl tonight call left number call back hr pleas help'
 'unit took aback respons flight attend put respons squar back passeng shoulder']


In [ ]:
#Convert textual data to numerical data
vectorizer=TfidfVectorizer() # assigns some importance to each word #checks if each word corresponds to +ve or -ve tweets

X_train=vectorizer.fit_transform(X_train) # understands vocab assigns score to each word
X_test=vectorizer.transform(X_test) # ensures test data is transformed using the same vocabulary and IDF values as training data. Ensures consistency and prevents overfitting.

In [ ]:
print(X_train)

  (0, 6261)	0.5643379776273547
  (0, 3102)	0.5741313193593448
  (0, 1220)	0.48362022840803653
  (0, 282)	0.3435219786457975
  (1, 8528)	0.6019977971884867
  (1, 9612)	0.3637892515311568
  (1, 2472)	0.38005358348655643
  (1, 2100)	0.27808481378562017
  (1, 965)	0.45620737778714227
  (1, 3093)	0.17179085678034475
  (1, 4538)	0.21411882834236964
  (2, 7318)	0.6044864596578694
  (2, 413)	0.6504362375134558
  (2, 8633)	0.3710895267850119
  (2, 9174)	0.27170090932882474
  (3, 177)	0.17719081969592407
  (3, 2532)	0.35292779299772287
  (3, 8269)	0.3187653831409321
  (3, 1862)	0.21120949062313718
  (3, 3631)	0.3425422492321636
  (3, 715)	0.3187653831409321
  (3, 5217)	0.29714220561592253
  (3, 9841)	0.17735914946263914
  (3, 8629)	0.25383500922392654
  (3, 2609)	0.3592980545174764
  :	:
  (13173, 2675)	0.5715937508154769
  (13173, 3090)	0.3619102896250965
  (13173, 8297)	0.5242394516232941
  (13173, 4538)	0.5171793921395951
  (13174, 6777)	0.6859904254438044
  (13174, 3552)	0.35826211426209276


In [ ]:
print(X_test)

  (0, 9612)	0.3487333721375469
  (0, 9290)	0.18548748300822226
  (0, 7769)	0.26599348836638514
  (0, 5433)	0.42872013643645684
  (0, 623)	0.7677408010295124
  (1, 9863)	0.39226236506765993
  (1, 8120)	0.17508075603857878
  (1, 7748)	0.35889359225875966
  (1, 4999)	0.3134818576305849
  (1, 3180)	0.3339264361428345
  (1, 2328)	0.3124634482373918
  (1, 633)	0.6158418700844485
  (2, 8528)	0.5339370927520948
  (2, 8516)	0.48818019879250946
  (2, 5562)	0.3086176701872625
  (2, 4920)	0.30330339155843433
  (2, 4538)	0.18991096851607583
  (2, 3093)	0.30473703079670683
  (2, 917)	0.40052793523820174
  (3, 8272)	0.37358253417836235
  (3, 7147)	0.30011184530875407
  (3, 6631)	0.28346592157358824
  (3, 4322)	0.36525957231077943
  (3, 4289)	0.4052246375800065
  (3, 3525)	0.18998893645157136
  :	:
  (1461, 1258)	0.47701332924752193
  (1462, 8846)	0.3013799752802201
  (1462, 6692)	0.23319624704862912
  (1462, 6134)	0.2781812047727463
  (1462, 4979)	0.2977514796583963
  (1462, 4029)	0.25271599363197716

In [ ]:
# Train ML model- Logistic Regression

model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
#Model Evaluation on training data

X_train_pred= model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train, X_train_pred)

In [ ]:
print('Accuracy of training data:' ,training_data_accuracy)

Accuracy of training data: 0.8674104432301154


In [ ]:
#Model Evaluation on test data

X_test_pred= model.predict(X_test)
test_data_accuracy= accuracy_score(Y_test, X_test_pred)

In [ ]:
print('Accuracy of testing data:' ,test_data_accuracy)


Accuracy of testing data: 0.7984972677595629


# Save the Trained Model

In [ ]:
import pickle

In [ ]:
filename= 'trained_model.sav'
pickle.dump(model, open(filename, 'wb')) #wb-> writing in binary format

In [ ]:
#Using saved model for future predictions
loaded_model= pickle.load(open('/content/trained_model.sav', 'rb'))

In [ ]:
X_new=X_test[600]
print(Y_test[600])

prediction= loaded_model.predict(X_new)
print(prediction)



negative
['negative']
